In [1]:
# TargetEncoder
#!pip install -q scikit-learn==1.4
# KerasClassifier
#!pip install -q --no-deps scikeras

In [1]:
#현재 커널에 설치가 안되어서 삽질..
#import sys
#import subprocess
# 현재 Python 실행 파일 경로를 사용하여 pip 설치 명령 실행
#subprocess.check_call([sys.executable, "-m", "pip", "install", "scikeras"])


In [2]:
import pandas as pd
import numpy as np
import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
from scikeras.wrappers import KerasClassifier

2024-07-25 23:37:12.185135: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Read and split the data
train_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')
train = train_origin.sample(frac=0.01, random_state = 42)

In [33]:
train = train_origin.sample(frac=0.05, random_state = 42).set_index('id').astype(str)

# 예측변수 분리 및 train, valid set 분리
X = train.drop(['Response'], axis=1)
y = train['Response']

# 훈련 세트와 테스트 세트로 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify = y, test_size=0.2, random_state=42)

# 전체 변수 타겟 인코딩
enc = TargetEncoder()
X_train = pd.DataFrame(enc.fit_transform(X_train, y_train), 
                       index=X_train.index, columns=X_train.columns)
X_valid = pd.DataFrame(enc.transform(X_valid), index=X_valid.index, 
                      columns=X_valid.columns)

In [22]:
results=[]

In [26]:
# The Keras model with two hidden layers
def get_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(64, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(256, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(32, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='he_normal', activation='sigmoid'))
    
    return model


relu_model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=0.003),
    validation_split=0.05,
    batch_size=1024,
    validation_batch_size=65536,
    epochs=20,
    callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3), keras.callbacks.EarlyStopping(patience=5)]
)

# Execute the pipeline and measure the auc score
relu_model.fit(X_train, y_train, verbose=0)
valid_preds = relu_model.predict_proba(X_valid)[:, 1]
valid_auc = roc_auc_score(y_valid, valid_preds)
results.append({'model':'relu', 'AUC':valid_auc})
print("ROC AUC:", valid_auc)

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
ROC AUC: 0.8670521083458771


In [27]:
# 모델 정의 함수
def get_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(64, kernel_initializer='he_normal', activation='gelu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='gelu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(256, kernel_initializer='he_normal', activation='gelu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='gelu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(32, kernel_initializer='he_normal', activation='gelu'))
    model.add(keras.layers.Dense(1, kernel_initializer='he_normal', activation='sigmoid'))
    
    return model

# 학습률 스케줄러 함수 정의
def lr_scheduler(epoch, lr):
    decay_rate = 0.96
    decay_step = 10
    new_lr = lr * (decay_rate ** (epoch // decay_step))
    print(f"Epoch {epoch}: Learning rate is {new_lr}")
    return new_lr

# LearningRateScheduler 콜백 설정
lr_schedule = LearningRateScheduler(lr_scheduler)

# Recreate the model with the best hyperparameters
gelu_model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=2e-03),
    validation_split=0.05,
    batch_size=128,
    validation_batch_size=65536,
    epochs=20,  # Increase the number of epochs for further training
    callbacks=[lr_schedule, keras.callbacks.EarlyStopping(patience=20)]
)

# Execute the pipeline and measure the auc score
gelu_model.fit(X_train, y_train, verbose=0)

valid_preds = gelu_model.predict_proba(X_valid)[:, 1]
valid_auc = roc_auc_score(y_valid, valid_preds)
results.append({'model':'gelu', 'AUC':valid_auc})
print("ROC AUC:", valid_auc)


Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6: Learning rate is 0.0020000000949949026
Epoch 7: Learning rate is 0.0020000000949949026
Epoch 8: Learning rate is 0.0020000000949949026
Epoch 9: Learning rate is 0.0020000000949949026
Epoch 10: Learning rate is 0.0019200000911951064
Epoch 11: Learning rate is 0.0018432000651955605
Epoch 12: Learning rate is 0.001769472062587738
Epoch 13: Learning rate is 0.00169869314879179
Epoch 14: Learning rate is 0.0016307454183697699
Epoch 15: Learning rate is 0.001565515547990799
Epoch 16: Learning rate is 0.0015028949081897734
Epoch 17: Learning rate is 0.001442779116332531
Epoch 18: Learning rate is 0.0013850679248571395
Epoch 19: Learning rate is 0.001329665221273899
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms

In [28]:
# 모델 정의 함수
def get_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(64, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(256, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(32, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.Dense(1, kernel_initializer='lecun_normal', activation='sigmoid'))
    
    return model

# 학습률 스케줄러 함수 정의
def lr_scheduler(epoch, lr):
    decay_rate = 0.96
    decay_step = 10
    new_lr = lr * (decay_rate ** (epoch // decay_step))
    print(f"Epoch {epoch}: Learning rate is {new_lr}")
    return new_lr

# LearningRateScheduler 콜백 설정
lr_schedule = LearningRateScheduler(lr_scheduler)

# Recreate the model with the best hyperparameters
selu_model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=2e-03),
    validation_split=0.05,
    batch_size=128,
    validation_batch_size=65536,
    epochs=20,  # Increase the number of epochs for further training
    callbacks=[lr_schedule, keras.callbacks.EarlyStopping(patience=20)]
)

# Execute the pipeline and measure the auc score
selu_model.fit(X_train, y_train, verbose=0)

valid_preds = selu_model.predict_proba(X_valid)[:, 1]
valid_auc = roc_auc_score(y_valid, valid_preds)
results.append({'model':'selu', 'AUC':valid_auc})
print("ROC AUC:", valid_auc)

Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6: Learning rate is 0.0020000000949949026
Epoch 7: Learning rate is 0.0020000000949949026
Epoch 8: Learning rate is 0.0020000000949949026
Epoch 9: Learning rate is 0.0020000000949949026
Epoch 10: Learning rate is 0.0019200000911951064
Epoch 11: Learning rate is 0.0018432000651955605
Epoch 12: Learning rate is 0.001769472062587738
Epoch 13: Learning rate is 0.00169869314879179
Epoch 14: Learning rate is 0.0016307454183697699
Epoch 15: Learning rate is 0.001565515547990799
Epoch 16: Learning rate is 0.0015028949081897734
Epoch 17: Learning rate is 0.001442779116332531
Epoch 18: Learning rate is 0.0013850679248571395
Epoch 19: Learning rate is 0.001329665221273899
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms

In [30]:
# 모델 정의 함수
def get_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(64, kernel_initializer='he_normal', activation='mish'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='mish'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(256, kernel_initializer='he_normal', activation='mish'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='mish'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(32, kernel_initializer='he_normal', activation='mish'))
    model.add(keras.layers.Dense(1, kernel_initializer='he_normal', activation='sigmoid'))
    
    return model
    
# 학습률 스케줄러 함수 정의
def lr_scheduler(epoch, lr):
    decay_rate = 0.96
    decay_step = 10
    new_lr = lr * (decay_rate ** (epoch // decay_step))
    print(f"Epoch {epoch}: Learning rate is {new_lr}")
    return new_lr

# LearningRateScheduler 콜백 설정
lr_schedule = LearningRateScheduler(lr_scheduler)

# Recreate the model with the best hyperparameters
mish_model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=2e-03),
    validation_split=0.05,
    batch_size=128,
    validation_batch_size=65536,
    epochs=20,  # Increase the number of epochs for further training
    callbacks=[lr_schedule, keras.callbacks.EarlyStopping(patience=20)]
)

# Execute the pipeline and measure the auc score
mish_model.fit(X_train, y_train, verbose=0)

valid_preds = mish_model.predict_proba(X_valid)[:, 1]
valid_auc = roc_auc_score(y_valid, valid_preds)
results.append({'model':'mish', 'AUC':valid_auc})
print("ROC AUC:", valid_auc)

Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6: Learning rate is 0.0020000000949949026
Epoch 7: Learning rate is 0.0020000000949949026
Epoch 8: Learning rate is 0.0020000000949949026
Epoch 9: Learning rate is 0.0020000000949949026
Epoch 10: Learning rate is 0.0019200000911951064
Epoch 11: Learning rate is 0.0018432000651955605
Epoch 12: Learning rate is 0.001769472062587738
Epoch 13: Learning rate is 0.00169869314879179
Epoch 14: Learning rate is 0.0016307454183697699
Epoch 15: Learning rate is 0.001565515547990799
Epoch 16: Learning rate is 0.0015028949081897734
Epoch 17: Learning rate is 0.001442779116332531
Epoch 18: Learning rate is 0.0013850679248571395
Epoch 19: Learning rate is 0.001329665221273899
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/

In [31]:
import subprocess

# 합성음을 재생하여 알림
subprocess.run(['say', '-v', 'Karen', 'Process complete'])

results

[{'model': 'relu', 'AUC': 0.8678594889055563},
 {'model': 'relu', 'AUC': 0.8670521083458771},
 {'model': 'gelu', 'AUC': 0.8665438270577911},
 {'model': 'selu', 'AUC': 0.8671588456642852},
 {'model': 'mish', 'AUC': 0.8659934080069389}]